In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import groupby  

To get the actual booking, probably look at MOST RECENT SCRAPE 

Scrapes contain data from the past as well as future, I will use this as assumed truth

In [4]:
listings = pd.read_pickle('data/scraped_listings_cleaned.pkl')

In [5]:
data = pd.read_pickle('data/scraped_data_cleaned.pkl')

In [11]:
df=data[data.scraping_id==2]


In [24]:
df.head()

,scraping_id,scraped_date,date,price,available
0,2,2018-03-15,2018-04-01,250,True
1,2,2018-03-15,2018-04-02,115,True
2,2,2018-03-15,2018-04-03,115,True
3,2,2018-03-15,2018-04-04,115,True
4,2,2018-03-15,2018-04-05,150,False


In [96]:
# Check where all scrapes after the booked date only have 1 unique value, aka they are thee same
df.loc[df.scraped_date >
        df.date].sort_values(by=['date','scraped_date'])\
         .groupby('date').mean()[
(~(df.loc[df.scraped_date >
        df.date].sort_values(by=['date','scraped_date'])\
         .groupby('date').nunique().eq(1).available))
    
]


,scraping_id,price,available
date,,,
2018-05-18,2,115,0.846154


In [202]:
data.price.unique()

array([ 250,  115,  150, ..., 1644, 1684, 9001])

Approach: 
Each date for each property is scraped N amount of times. During each of these scrapes, the availablility of the specific date can change. 

1. First assumption is that if the last scraped value contains a True in the availablility column, it was not booked

2 Next issue is the False availibility. 

- For the list of scraped dates, we're making an assumption that any time there is a true in the available column that later turns to false, the date was booked for revenue. The assumption here is that blocked dates would be known in advance and made earlier .

- There is an interesting to note with price. Sometimes the day is available at a higher price, then price drops until the day is registered as not available. This is highly indicative of a booking and a price meeting demand. Perhaps this could be captured by comparing to the first scrape of the booking date


reference: https://blog.pillow.com/crunching-the-numbers-defining-normal-for-airbnb-hosts-and-listings/

do a diff of first and last value in constant date, variable scraped_date table

F-T-F -possible booking
first scrape - last scrape :

T-F = booking?
F-F = blocked



Implementation

new table with 'ended_on' category

In [55]:
df.loc[(df.date == pd.datetime(2018,5,18)) & (df.scraped_date > df.date)]

,scraping_id,scraped_date,date,price,available
3419,2,2018-05-19,2018-05-18,115,False
3452,2,2018-05-20,2018-05-18,115,False
3485,2,2018-05-21,2018-05-18,115,True
3518,2,2018-05-22,2018-05-18,115,True
3551,2,2018-05-23,2018-05-18,115,True
3584,2,2018-05-24,2018-05-18,115,True
3617,2,2018-05-25,2018-05-18,115,True
3650,2,2018-05-26,2018-05-18,115,True
3683,2,2018-05-27,2018-05-18,115,True
3716,2,2018-05-28,2018-05-18,115,True


In [191]:
df['scraping_id'].value_counts()

2    3829
Name: scraping_id, dtype: int64

In [192]:
set(df.groupby('date').scraped_date.count())

{46, 77}

In [198]:
listings.groupby(by=['host_name','is_superhost']).count()['scraping_id'].sort_values(ascending=False)

host_name               is_superhost
Evolve Vacation Rental  False           335
Stay Alfred             False           153
Casago                  False           128
Chelsea                 False           122
Leslie                  False            68
Ardent                  False            59
Scott                   False            59
John                    False            58
Barbara                 False            57
Red Awning              False            51
WanderJaunt             True             51
Michael                 False            48
Elite Vacation Rentals  False            48
Ryan                    False            48
Meridian CondoResorts   False            43
David                   False            43
Scottsdale              False            40
Michael                 True             40
Rachel                  False            36
Jennifer                False            35
Evan                    True             34
Jeff                    False          

In [185]:
data.head()

,scraping_id,scraped_date,date,price,available
0,2,2018-03-15,2018-04-01,250,True
1,2,2018-03-15,2018-04-02,115,True
2,2,2018-03-15,2018-04-03,115,True
3,2,2018-03-15,2018-04-04,115,True
4,2,2018-03-15,2018-04-05,150,False


In [223]:
pd.set_option('display.max_rows', 500)
data[data.scraping_id.isin([48]) & (data.date==pd.datetime(2018,5,9))]

,scraping_id,scraped_date,date,price,available
149318,48,2018-03-15,2018-05-09,99,True
149379,48,2018-03-16,2018-05-09,99,True
149440,48,2018-03-17,2018-05-09,99,True
149501,48,2018-03-18,2018-05-09,99,True
149562,48,2018-03-19,2018-05-09,99,True
149623,48,2018-03-20,2018-05-09,99,True
149684,48,2018-03-21,2018-05-09,99,True
149745,48,2018-03-22,2018-05-09,99,True
149806,48,2018-03-23,2018-05-09,99,True
149867,48,2018-03-24,2018-05-09,99,True


In [222]:
data[data.scraping_id.isin(list(range(45,50))) & (data.scraped_date==data.scraped_date.max())]

,scraping_id,scraped_date,date,price,available
141655,45,2018-05-31,2018-04-29,84,False
141656,45,2018-05-31,2018-04-30,84,False
141657,45,2018-05-31,2018-05-01,84,False
141658,45,2018-05-31,2018-05-02,84,False
141659,45,2018-05-31,2018-05-03,84,False
141660,45,2018-05-31,2018-05-04,84,False
141661,45,2018-05-31,2018-05-05,84,False
141662,45,2018-05-31,2018-05-06,84,False
141663,45,2018-05-31,2018-05-07,84,False
141664,45,2018-05-31,2018-05-08,84,False


In [ ]:
commercial 

# Coding Challenge
Set up columns in data df

1. Find max date that each was scraped
    -groupby [scraping_id, booking_date], max scraped_date
- Make new occupied_df that has data df but drop all rows that scraped date does not equal to max_scraped_date

New column in the occupied_df, **was_occupied**

For each one day in each property (should be one row)
1. access the last scraped available value for it in data_df
    - If the last scraped available value was True, ignore (Never got booked)
- If the last scraped available was False:
    - Open all scraped dates for that one booking date
    - Check the sum of true false trues
        - if sum is greater than 0, set was_occupied column to True
            - This suggests that at some point, the booking was available


In [263]:
pd.__version__

'0.24.2'

In [352]:
def remove_consec_dups(input_list): 
    return [i[0] for i in groupby(input_list)] 


In [318]:
def was_occupied(row):
    if row['available']:
        return False
    if row['available_sum'] >= 1:
        return True
    else:
        return False

In [410]:
def find_available_counts(df):
    df_available = df.groupby(['scraping_id','date'])\
            .agg({'scraped_date':'max','available':['sum','count']})
    df_available.columns = ['scraped_date','available_sum','available_count']
    df_merged=df.merge(df_available,on=['scraping_id','scraped_date','date'])
    price_availability = df.groupby(['scraping_id','date']).agg({'price':lambda x: (remove_consec_dups(x)),
                                        'available':lambda x: (remove_consec_dups(x))})
    price_availability.columns = ['prices','availability_states']
    df_merged = df_merged.merge(price_availability,on=['scraping_id','date'])
    df_merged['was_occupied'] = df_merged.apply(was_occupied,axis = 1)
    return df_merged

In [412]:
df_test = find_available_counts(df)

In [415]:
df_test.head()

,scraping_id,scraped_date,date,price,available,available_sum,available_count,prices,availability_states,was_occupied
0,2,2018-04-30,2018-04-01,250,True,46.0,46,[250],[True],False
1,2,2018-04-30,2018-04-02,115,False,15.0,46,[115],"[True, False]",True
2,2,2018-04-30,2018-04-03,115,True,46.0,46,[115],[True],False
3,2,2018-04-30,2018-04-04,115,True,46.0,46,[115],[True],False
4,2,2018-04-30,2018-04-05,145,False,0.0,46,"[150, 145]",[False],False


In [422]:
df_test.sample(10)

,scraping_id,scraped_date,date,price,available,available_sum,available_count,prices,availability_states,was_occupied
35,2,2018-05-31,2018-05-06,115,True,77.0,77,"[150, 133, 132, 131, 115]",[True],False
49,2,2018-05-31,2018-05-20,115,True,11.0,77,"[150, 115]","[False, True]",False
45,2,2018-05-31,2018-05-16,115,False,59.0,77,"[150, 135, 136, 135, 115, 100, 115]","[True, False]",True
11,2,2018-04-30,2018-04-12,115,False,0.0,46,"[150, 115]",[False],False
17,2,2018-04-30,2018-04-18,115,False,0.0,46,"[150, 128, 127, 126, 127, 115]",[False],False
32,2,2018-05-31,2018-05-03,115,False,0.0,77,"[150, 115]",[False],False
24,2,2018-04-30,2018-04-25,115,False,0.0,46,"[150, 115]",[False],False
54,2,2018-05-31,2018-05-25,118,False,26.0,77,"[150, 138, 137, 138, 142, 141, 115, 122, 123, ...","[True, False]",True
38,2,2018-05-31,2018-05-09,115,False,36.0,77,"[150, 136, 134, 115]","[True, False]",True
55,2,2018-05-31,2018-05-26,115,True,77.0,77,"[150, 147, 143, 146, 154, 148, 120, 128, 124, ...",[True],False


In [421]:
df_test.was_occupied.sum()/(~df_test.available).sum()

0.4

In [425]:
(df_test['price'] * df_test['was_occupied']).sum()

1843

In [426]:
listings

,scraping_id,listing,city,lon,lat,mapped_location,name,capacity,bathrooms,bedrooms,has_pool,cleaning_fee,is_superhost,host_name
0,2,https://www.airbnb.com/rooms/17720788,Scottsdale,-111.916935,33.496566,https://maps.google.com/maps?q=33.496565678911...,Old Town Scottsdale Vacation Home & GIANTS Rental,7,2.5,2,True,150.0,True,Mj
1,3,https://www.airbnb.com/rooms/4400911,Scottsdale,-111.898977,33.463182,https://maps.google.com/maps?q=33.463182488027...,Upscale Townhouse South Scottsdale,4,1.5,2,True,85.0,True,Jamil
2,4,https://www.airbnb.com/rooms/19748678,Scottsdale,-111.957684,33.607810,https://maps.google.com/maps?q=33.607809726520...,North Scottsdale/ PV Private Guest House w/pool,4,1,0,True,75.0,True,Julie And Bill
3,5,https://www.airbnb.com/rooms/18900365,Scottsdale,-111.915536,33.499397,https://maps.google.com/maps?q=33.499396773678...,Mountain View Paradise*Heated Pool*Spa*Gym,4,1,1,True,100.0,True,Vanessa
4,6,https://www.airbnb.com/rooms/15808110,Scottsdale,-111.910657,33.490571,https://maps.google.com/maps?q=33.490570685645...,Modern Luxury Scottsdale Condo!,5,1,2,True,150.0,True,Sara
5,7,https://www.airbnb.com/rooms/16904175,Scottsdale,-111.905422,33.510514,https://maps.google.com/maps?q=33.510514039976...,Complete retreat with private hot tub.,6,2,2,True,75.0,False,Larry
6,8,https://www.airbnb.com/rooms/17478523,Scottsdale,-111.935345,33.502713,https://maps.google.com/maps?q=33.502712546830...,Stay in Old Town - Next to Fashion Square Mall!,4,1.5,2,True,110.0,True,Arizona
7,9,https://www.airbnb.com/rooms/16217702,Scottsdale,-111.934066,33.491852,https://maps.google.com/maps?q=33.491852335222...,WALK TO OLD TOWN- SLEEPS 6!,6,1.5,2,True,80.0,False,Leslie
8,10,https://www.airbnb.com/rooms/18488203,Scottsdale,-111.903139,33.639273,https://maps.google.com/maps?q=33.639272816775...,SCOTTSDALE PARADISE,6,2,2,True,90.0,True,Vince
9,11,https://www.airbnb.com/rooms/20012732,Scottsdale,-111.898074,33.508856,https://maps.google.com/maps?q=33.508856187252...,NEW! Remodeled Chic Townhome near Old Town!,6,2.5,2,True,100.0,False,Shayna


In [301]:
data_available=find_available_counts(data)

In [326]:
listings.bathrooms.astype(float).unique()

array([ 2.5,  1.5,  1. ,  2. ,  3. ,  3.5,  5. ,  4.5,  4. ,  5.5,  7. ,
        7.5,  6. ,  9.5,  8. ,  6.5,  nan,  0.5, 10. ,  9. ,  8.5,  0. ])